In [1]:
from data_prep import *
from charts import *
import pandas as pd


# Squad 

In [2]:
squads_by_season(1, min_games=5).save("Charts/1s-squad.html")
squads_by_season(2, min_games=5).save("Charts/2s-squad.html")

plot_starts_by_position(1, season="2023/24", fb_only=True).save("Charts/2023-24/1s-squad-fb.html")
plot_starts_by_position(1, season="2023/24", fb_only=False).save("Charts/2023-24/1s-squad-position.html")
plot_starts_by_position(2, season="2023/24", fb_only=False).save("Charts/2023-24/2s-squad-position.html")

plot_games_by_player(0, min_games=10).save("Charts/2023-24/appearances.html")
plot_games_by_player(1, min_games=5).save("Charts/2023-24/1s-appearances.html")
plot_games_by_player(2, min_games=5).save("Charts/2023-24/2s-appearances.html")

team_of_the_season_chart(1, "2023/24").save("Charts/2023-24/1s-lineup.html")
team_of_the_season_chart(2, "2023/24").save("Charts/2023-24/2s-lineup.html")

points_scorers_chart(1).save("Charts/2023-24/1s-points.html")
points_scorers_chart(2).save("Charts/2023-24/2s-points.html")

captains_chart().save("Charts/2023-24/captains.html")
card_chart().save("Charts/2023-24/cards.html")

lineout_chart(1, "2023/24").save("Charts/2023-24/1s-lineouts.html")
lineout_chart(2, "2023/24").save("Charts/2023-24/2s-lineouts.html")

results_chart(1, "2023/24").save("Charts/2023-24/1s-results.html")
results_chart(2, "2023/24").save("Charts/2023-24/2s-results.html")

/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:276: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mcp = df.groupby(["Player", by]).size().reset_index(name="Count")\
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:278: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(by).head(4).reset_index(drop=True)\
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:308: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt

# 🎯 Lineouts


In [13]:
count_success_chart("Numbers", 1).save("Charts/1s-numbers.html")
count_success_chart("Area", 1).save("Charts/1s-area.html")
count_success_chart("Jumper", 1, min=10).save("Charts/1s-jumper.html")
count_success_chart("Hooker", 1, min=10).save("Charts/1s-hooker.html")
count_success_chart("Call", 1, min=10).save("Charts/1s-call.html")
count_success_chart("Dummy", 1).save("Charts/1s-dummy.html")
# (
#     count_success_chart("Numbers", 1) 
#     & count_success_chart("Area", 1) 
#     & count_success_chart("Dummy", 1)
#     & count_success_chart("Jumper", 1, min=5)
# ).resolve_scale(color="independent")


In [14]:
count_success_chart("Jumper", 1)
# count_success_chart("Hooker", 1, min=5)

alt.FacetChart(...)

In [10]:
count_success_chart("Call", 1, min=10).save("Charts/1s-call.html")

alt.FacetChart(...)

# Results